In [1]:
# Manipulación de Datos

In [2]:
## Lectura de archivo

In [3]:
diamonds = sqlContext.read.format('com.databricks.spark.csv').options(
header='true', inferSchema='true').load('/databricks-datasets/Rdatasets/data-001/csv/ggplot2/diamonds.csv')

In [4]:
display(diamonds)

In [5]:
diamonds.printSchema()

In [6]:
print(diamonds.count())

In [7]:
display(diamonds.select("color").distinct().collect())

In [8]:
## Calculando los valores - precio / carat

In [9]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import *
diamondsCast = diamonds.withColumn("price", diamonds["price"].cast(DoubleType()))
carat_avgPrice = (diamondsCast
               .groupBy("carat")
               .avg("price")
               .withColumnRenamed("avg(price)", "avgPrice")
               .orderBy(desc("avgPrice")))
carat_avgPrice.show(10)

In [10]:
### No sirve de mucho hacer un agrupado por una columna con valores continuos, así que podemos redondear un poco:
diamondsCast2= diamondsCast.withColumn("carat2",round(col("carat")))
diamondsCast2.show(10)

In [11]:
(diamondsCast2.groupBy("carat2").avg("price").withColumnRenamed("avg(price)", "avgPrice").orderBy(desc("avgPrice"))).show()

In [12]:
# También podríamos obtener el máximo de tabla por corte
from pyspark.sql.window import Window
import pyspark.sql.functions as func
windowSpec = Window.partitionBy(diamondsCast['cut']).orderBy(diamondsCast['table'].desc())
diamondsCast.select("cut","table",func.row_number().over(windowSpec).alias('lugar')).filter("lugar=1").show()

In [13]:
diamDf= diamondsCast.rdd
# Suma del precio por claridad
claridadPrecio= diamDf.map(lambda x: (x.clarity,x.price)).reduceByKey(lambda x,y:x+y)
display(claridadPrecio.collect())